In [ ]:
import cv2
import numpy as np
import serial
import numpy as np
import time

In [ ]:
# 这一步用于识别黑色条纹的位置，计算误差后传给stm32
def get_road_error(line):
    # 100:540
    d=np.argwhere(line[70:170,100:540] < 250)
    dis=np.mean(d,axis=0)-220
    k=dis[1]
    k=min(max(-9000,k),9000)
    x=0.8
    if k>=0:
        error='E'+'-'+str(int(int(k**(x))%1000/100))+str(int(int(k**(x))%100/10))+str(int(k**(x))%10)
    if k<0:
        error='E'+'+'+str(int(int((-k)**(x))%1000/100))+str(int(int((-k)**(x))%100/10))+str(int((-k)**(x))%10)
    print(error)
    return error,k

# 对图片进行分割操作, 分别返回黑色与红色的图片
def photo_process(cap):
    _,image=cap.read()
    red=np.abs(image[:,:,2].astype(np.float32)-0.5*(image[:,:,0].astype(np.float32)+image[:,:,1].astype(np.float32)))
    black = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # black = cv2.Canny(black, 240, 200)
    # 对红色图片进行二值化，阈值为30
    _, red = cv2.threshold(red, 45, 255, cv2.THRESH_BINARY)
    _, black= cv2.threshold(black, 100, 255, cv2.THRESH_BINARY)
    return red, black

# 1.测试路口弯道识别
# 在路口识别分支，有分支为0，正常为1
def get_cross(road):
    if np.sum(road[:,0:250] > 250)>300:
        return 0
    else:
        return 1

# 2.测试停车标志识别
# 监测停止标志, 有为0，没有为1
def get_stop(road):
    if (np.sum(road[:,0:250] > 250)>100)&(np.sum(road[:,389:639] > 250)>100):
        return 0
    else:
        return 1

# 3.测试车辆识别
# 看前面有没有2车，有为1，没有为0
def see_car2(red):
    if np.sum(red[0:240,200:439] > 250)>200:
        return 1
    else:
        return 0

In [ ]:
cap1=cv2.VideoCapture(1)
# cap2=cv2.VideoCapture(2)

In [ ]:
line=cv2.imread('outcornor.jpg')
line_grey=cv2.cvtColor(line, cv2.COLOR_BGR2GRAY)
_, line_grey= cv2.threshold(line_grey, 100, 255, cv2.THRESH_BINARY)
# 200 440
d=np.argwhere(line_grey[70:170,100:540] < 250)
dis=np.mean(d,axis=0)-220
k=dis[1]
k=min(max(-10000,k),10000)
x=0.8
if k>=0:
    error='E'+'-'+str(int(int(k**(x))%1000/100))+str(int(int(k**(x))%100/10))+str(int(k**(x))%10)
if k<0:
    error='E'+'+'+str(int(int((-k)**(x))%1000/100))+str(int(int((-k)**(x))%100/10))+str(int((-k)**(x))%10)
print(error)

In [ ]:
np.mean(d,axis=0)

In [ ]:
while True:
    _,image=cap1.read()
    red, black = photo_process(cap1)
    get_road_error(black)
    cv2.imshow('black',black)
    cv2.imshow('image',image)
    cv2.waitKey(1)